In [16]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

DATA_WATER = Path("Water Access Data.csv")
DATA_SEWER = Path("Sewer Access Data.csv")
OUT_DIR = Path("Outputs")

In [ ]:
def load_data() -> pd.DataFrame:
    """Load, clean, and combine water + sewer CSVs into one DataFrame."""
    water_df = pd.read_csv(DATA_WATER)
    sewer_df = pd.read_csv(DATA_SEWER)

    water_df.columns = water_df.columns.str.replace(r'^(w_|s_)', '', regex=True)
    sewer_df.columns = sewer_df.columns.str.replace(r'^(w_|s_)', '', regex=True)

    df = pd.concat([water_df, sewer_df], ignore_index=True)
    df.columns = df.columns.str.replace(r'^(w_|s_)', '', regex=True)

    # print columns
    print("Data columns:", df.columns.tolist())

    # Dtypes & minor hygiene
    if "year" in df.columns:
        df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    if "zone" in df.columns:
        df["zone"] = df["zone"].astype("string").str.strip()
    if "country" in df.columns:
        df["country"] = df["country"].astype("string").str.strip()
    if "type" in df.columns:
        df["type"] = (
            df["type"]
            .astype("string")
            .str.strip()
            .str.lower()
            .replace({"w_access": "water", "s_access": "sewer"})
        )

    return df
def _pct_fmt(v):
    return None if pd.isna(v) else f"{v:.1f}%"

def _ensure_year_int(df):
    if "year" in df.columns:
        df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    return df

def _country_summary_2024(df: pd.DataFrame) -> pd.DataFrame:
    """
    One row per (country, type) in 2024 with summary stats needed for text ranges.
    """
    d = df.copy()
    d = d[d["year"] == 2024]
    if "type" not in d.columns:
        # fallback: treat all as 'water' context if 'type' missing
        d["type"] = "unknown"

    # sewer gap = unimproved + open_def (sewer)
    d["sewer_gap_pct"] = d.get("unimproved_pct", np.nan) + d.get("open_def_pct", np.nan)

    # aggregate (min/median/max) per country, type
    agg = (
        d.groupby(["country", "type"])
        .agg(
            safely_min=("safely_managed_pct", "min"),
            safely_med=("safely_managed_pct", "median"),
            safely_max=("safely_managed_pct", "max"),
            open_def_min=("open_def_pct", "min"),
            open_def_med=("open_def_pct", "median"),
            open_def_max=("open_def_pct", "max"),
            unimproved_min=("unimproved_pct", "min"),
            unimproved_med=("unimproved_pct", "median"),
            unimproved_max=("unimproved_pct", "max"),
            sewer_gap_min=("sewer_gap_pct", "min"),
            sewer_gap_med=("sewer_gap_pct", "median"),
            sewer_gap_max=("sewer_gap_pct", "max"),
            zones=("zone", "nunique"),
            popn_sum=("popn_total", "sum"),
        )
        .reset_index()
    )
    return agg

def _surface_water_2024(df: pd.DataFrame) -> pd.DataFrame:
    """
    2024, water-type: compute pct and estimated counts by zone, plus country ranges.
    """
    d = df.copy()
    d = d[(d["year"] == 2024) & (d["type"] == "water")]
    d = d.copy()
    if "surface_water_pct" not in d.columns:
        d["surface_water_pct"] = np.nan
    if "popn_total" not in d.columns:
        d["popn_total"] = np.nan
    # Keep only rows with valid values for water-specific metrics
    d = d.dropna(subset=["surface_water_pct", "popn_total"]).copy()
    d["surface_users_est"] = (d["surface_water_pct"] / 100.0) * d["popn_total"]

    # country range summary
    rng = (
        d.groupby("country")
        .agg(
            pct_min=("surface_water_pct", "min"),
            pct_med=("surface_water_pct", "median"),
            pct_max=("surface_water_pct", "max"),
            users_min=("surface_users_est", "min"),
            users_med=("surface_users_est", "median"),
            users_max=("surface_users_est", "max"),
        )
        .reset_index()
    )
    return d, rng

def _trend_series(df: pd.DataFrame) -> pd.DataFrame:
    """Return 2020–2024 rows for time-series plots."""
    d = df.copy()
    d = d[(d["year"] >= 2020) & (d["year"] <= 2024)]
    return d

def _urban_rural_tag(zone: str) -> str:
    if pd.isna(zone):
        return "unknown"
    z = str(zone).lower()
    if "rural" in z:
        return "rural"
    if "urban" in z:
        return "urban"
    # heuristics for named capitals; extend as needed
    if any(k in z for k in ["yaounde", "douala", "kawempe", "kampala", "maseru", "lilongwe", "blantyre"]):
        return "urban"
    return "other"

def build_dashboard(df: pd.DataFrame) -> Path:
    df = _ensure_year_int(df)

    # --------------- 1) OVERALL COMPARISON (2024) ----------------
    s24 = _country_summary_2024(df)

    # Safely managed by country & type (median with range ribbons)
    sm = s24.dropna(subset=["safely_med"]).copy()
    fig_overall = px.bar(
        sm,
        x="country",
        y="safely_med",
        color="type",
        barmode="group",
        hover_data={
            "safely_med": ":.1f",
            "safely_min": ":.1f",
            "safely_max": ":.1f",
            "open_def_med": ":.1f",
            "unimproved_med": ":.1f",
            "zones": True,
            "popn_sum": True,
        },
        title="2024 Safely Managed Coverage by Country (Median with min/max shown in hover)",
    )
    fig_overall.update_layout(yaxis_title="Safely Managed % (median)")

    # --------------- 2) CRITICAL KPI #1: Sewer Access Gap (Unimproved + Open Def) -------------
    gap = s24[s24["type"] == "sewer"].dropna(subset=["sewer_gap_med"]).copy()
    fig_gap = px.bar(
        gap,
        x="country",
        y="sewer_gap_med",
        hover_data={"sewer_gap_min": ":.1f", "sewer_gap_max": ":.1f"},
        title="2024 Sewer Access Gap (Unimproved + Open Defecation) — Median by Country",
    )
    fig_gap.update_layout(yaxis_title="Gap % (median)")

    # --------------- 3) CRITICAL KPI #2: Surface Water Exposure (Water type, 2024) -------------
    sw_2024, sw_rng = _surface_water_2024(df)
    fig_surface_pct = px.strip(
        sw_2024,
        x="country",
        y="surface_water_pct",
        hover_data=["zone", "surface_water_pct", "popn_total", "surface_users_est"],
        title="2024 Surface Water Users by Zone (pct)",
    )
    # counts scatter for intuition
    fig_surface_cnt = px.scatter(
        sw_2024,
        x="country",
        y="surface_users_est",
        size="popn_total",
        hover_data=["zone", "surface_water_pct", "popn_total", "surface_users_est"],
        title="2024 Estimated Surface Water Users by Zone (count; bubble ~ population)",
    )
    fig_surface_cnt.update_layout(yaxis_title="Estimated users")

    # --------------- 4) KPI #3: Population Coverage Trends (2020–2024) -------------------------
    ts = _trend_series(df)
    fig_pop_trend = px.line(
        ts.groupby(["country", "year"], as_index=False)["popn_total"].sum(),
        x="year",
        y="popn_total",
        color="country",
        markers=True,
        title="Population Trend (2020–2024)",
    )

    # --------------- 5) KPI #4: Urban vs Rural Disparities ------------------------------------
    # Tag zones and show safely managed water & sewer side-by-side for 2024
    ur = df[df["year"] == 2024].copy()
    ur["ur_tag"] = ur["zone"].map(_urban_rural_tag)

    # Lesotho zoom-in
    les = ur[ur["country"].str.upper() == "LESOTHO"].copy()
    fig_les = px.bar(
        les,
        x="zone",
        y="safely_managed_pct",
        color="type",
        hover_data=["ur_tag", "open_def_pct", "unimproved_pct"],
        title="Lesotho 2024 — Safely Managed by Zone (Urban vs Rural tag in hover)",
    )
    fig_les.update_layout(xaxis_tickangle=-30, yaxis_title="Safely Managed %")

    # Malawi zoom-in (weakest sewer)
    mw = ur[ur["country"].str.upper() == "MALAWI"].copy()
    fig_mw = px.bar(
        mw,
        x="zone",
        y="safely_managed_pct",
        color="type",
        hover_data=["ur_tag", "open_def_pct", "unimproved_pct"],
        title="Malawi 2024 — Safely Managed by Zone",
    )
    fig_mw.update_layout(xaxis_tickangle=-30, yaxis_title="Safely Managed %")

    # --------------- 6) KPI #5: YoY Trends by Zone (selected highlights) -----------------------
    # Cameroon Yaounde*, Lesotho Maseru*, Uganda Kawempe, Malawi all zones
    focus_zones = ts[
        ts["zone"].str.contains("yaounde|maseru|kawempe", case=False, na=False)
        | ts["country"].str.upper().isin(["MALAWI"])
    ].copy()

    fig_yoy = px.line(
        focus_zones,
        x="year",
        y="safely_managed_pct",
        color="zone",
        facet_row="country",
        facet_col="type",
        markers=True,
        title="YoY Safely Managed by Zone (2020–2024) — Focused Zones",
        hover_data=["country", "zone", "type"],
    )
    fig_yoy.update_layout(yaxis_title="Safely Managed %")

    # --------------- 7) Priority Zones table (2024 snapshot) -----------------------------------
    pri = (
        ur.assign(sewer_gap_pct=lambda x: x.get("unimproved_pct", np.nan) + x.get("open_def_pct", np.nan))
        .loc[
            lambda x: (
                (x["country"].str.upper() == "MALAWI")
                | (x["zone"].str.contains("kawempe", case=False, na=False))
                | (x["zone"].str.contains("yaounde 1", case=False, na=False))
                | ((x["country"].str.upper() == "LESOTHO") & (x["zone"].str.contains("rural", case=False, na=False)))
            )
        ][
            ["country", "zone", "type", "popn_total", "safely_managed_pct", "open_def_pct", "unimproved_pct", "sewer_gap_pct"]
        ]
        .sort_values(["country", "zone", "type"])
    )

    # Make a small HTML table
    pri_tbl_html = (
        pri.rename(
            columns={
                "popn_total": "population",
                "safely_managed_pct": "safely_managed_%",
                "open_def_pct": "open_def_%",
                "unimproved_pct": "unimproved_%",
                "sewer_gap_pct": "sewer_gap_%",
            }
        )
        .to_html(index=False, float_format=lambda v: f"{v:.1f}")
    )

    # --------------- Assemble a single HTML artifact -------------------------------------------
    parts = []
    for title, fig in [
        ("Overall_2024", fig_overall),
        ("Sewer_Gap_2024", fig_gap),
        ("Surface_Pct_2024", fig_surface_pct),
        ("Surface_Counts_2024", fig_surface_cnt),
        ("Population_Trends", fig_pop_trend),
        ("Lesotho_UR_2024", fig_les),
        ("Malawi_UR_2024", fig_mw),
        ("YoY_Focus_Zones", fig_yoy),
    ]:
        parts.append(pio.to_html(fig, include_plotlyjs=False, full_html=False, config={"displayModeBar": True}))

    html = f"""
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Water & Sanitation KPIs — Interactive Dashboard</title>
  <script src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>
  <style>
    body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; margin: 16px; }}
    h1 {{ margin: 0 0 8px 0; }}
    h2 {{ margin-top: 28px; }}
    .card {{ margin: 18px 0; padding: 12px; border: 1px solid #eee; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.04); }}
    .note {{ color: #555; }}
    table {{ border-collapse: collapse; width: 100%; }}
    th, td {{ border: 1px solid #ddd; padding: 6px 8px; }}
    th {{ background: #fafafa; text-align: left; }}
  </style>
</head>
<body>
  <h1>Water & Sanitation KPIs — Interactive Dashboard</h1>
  <p class="note">Data scope: 2020–2024 (your combined CSVs). Hover on points/bars for zone-level details.</p>

  <div class="card"><h2>Overall comparison by country (2024)</h2>{parts[0]}</div>
  <div class="card"><h2>Critical KPI #1 — Sewer access gap (2024)</h2>{parts[1]}</div>
  <div class="card"><h2>Critical KPI #2 — Surface water exposure (pct by zone, 2024)</h2>{parts[2]}</div>
  <div class="card"><h2>Critical KPI #2 — Estimated surface water users (counts by zone, 2024)</h2>{parts[3]}</div>
  <div class="card"><h2>Population coverage trends (2020–2024)</h2>{parts[4]}</div>
  <div class="card"><h2>Urban vs rural disparities — Lesotho (2024)</h2>{parts[5]}</div>
  <div class="card"><h2>Urban vs rural disparities — Malawi (2024)</h2>{parts[6]}</div>
  <div class="card"><h2>Year-over-year safely managed by zone — Focused zones (2020–2024)</h2>{parts[7]}</div>

  <div class="card">
    <h2>Top priority zones for intervention (2024 snapshot)</h2>
    {pri_tbl_html}
    <p class="note">Sewer gap = unimproved % + open defecation % (sewer). Use column header tooltips and filters in your source tools for deeper dives.</p>
  </div>
</body>
</html>
    """.strip()

    out_path = OUT_DIR / "kpi_dashboard.html"
    out_path.write_text(html, encoding="utf-8")
    print(f"Dashboard saved to: {out_path}")
    return out_path

df = load_data()
print(df.tail())
build_dashboard(df)

Data columns: ['country', 'zone', 'year', 'safely_managed', 'safely_managed_pct', 'basic', 'basic_pct', 'limited', 'limited_pct', 'unimproved', 'unimproved_pct', 'surface_water', 'surface_water_pct', 'other_pct', 'popn_total', 'households', 'municipal_coverage', 'type', 'open_def', 'open_def_pct']
    country          zone  year  safely_managed  safely_managed_pct   basic  \
175  malawi      Lumbadzi  2023            3768               22.64    4947   
176  malawi      Old Town  2024          202184               27.99  184944   
177  malawi  Capital Hill  2024          138150               26.66  155144   
178  malawi       Kanengo  2024            9613               21.63   12410   
179  malawi      Lumbadzi  2024            3792               21.85    5873   

     basic_pct  limited  limited_pct  unimproved  unimproved_pct  \
175      29.71     3864        23.21        3191           19.17   
176      25.61   170794        23.65      133249           18.45   
177      29.94   11989

PosixPath('Outputs/kpi_dashboard.html')

# things to visualize
  - both datatsets
   - basic, limited, unimproved (show with scatter, and box plot)
  - In sewer access
    - open_def, surface_water, municipal_coverage (show as time_series)